In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops

print(tf.__version__)

2.1.0


### n-mode product in TensorFlow

In [2]:
def n_mode_product(x, u, n):# x, uはデータ型がfloat32である必要がある
    n = int(n)
    # We need one letter per dimension
    # (maybe you could find a workaround for this limitation)
    if n > 26:
        raise ValueError('n is too large.')
    ind = ''.join(chr(ord('a') + i) for i in range(n))
    exp = f'{ind}K...,JK->{ind}J...'
    return tf.einsum(exp, x, u)

In [3]:
x = tf.ones((2, 3, 4, 5)) # x, uはデータ型がfloat32である必要がある
u1 = tf.ones((1, 4))
u2 = tf.ones((1, 5))
n = 2  # n is zero-based here
out = n_mode_product(n_mode_product(x, u1, 2), u2, 3)
print(out.shape)

(2, 3, 1, 1)


In [6]:
out

<tf.Tensor: shape=(2, 3, 1, 1), dtype=float32, numpy=
array([[[[20.]],

        [[20.]],

        [[20.]]],


       [[[20.]],

        [[20.]],

        [[20.]]]], dtype=float32)>

In [7]:
tf.reshape(out, [2,3])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[20., 20., 20.],
       [20., 20., 20.]], dtype=float32)>

## TeSIA実装

In [367]:
class CustomLoss(tf.keras.losses.Loss):
    def __init__(self, name="custom_loss", C = 1, epsilon = 0.5, **kwargs):
        super(CustomLoss, self).__init__(name=name, **kwargs)
        self.C = C
        self.epsilon = epsilon

    def call(self, y_true, y_pred):
        y_pred = ops.convert_to_tensor(y_pred)  # 念のためTensor化
        y_true = math_ops.cast(y_true, y_pred.dtype) # 念のため同じデータ型化
        loss = self.C*K.mean(K.maximum(0., K.abs(y_true - y_pred) - self.epsilon))
        #loss = self.C*K.mean(K.abs(y_true - y_pred))
        #loss = self.C*K.mean(K.square(y_true - y_pred))
        return loss**2

# エポックが終わるごとにドットを一つ出力することで進捗を表示
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

### 3階テンソルの時系列データのみ
### n_mode_product関数を定義しておくこと
class TeSIA(object): 
    def __init__(self, epsilon=0.5, C=1):
        self.epsilon = epsilon
        self.C = C
    
    def mk_kernel_com(self, shape, dtype=None): 
        return K.constant(self.W_com, shape = shape, dtype=dtype)
    
    def mk_kernel_eve(self, shape, dtype=None): 
        return K.constant(self.W_eve, shape = shape, dtype=dtype)
    
    def mk_kernel_sen(self, shape, dtype=None): 
        return K.constant(self.W_sen, shape = shape, dtype=dtype)
    
    def mk_bias(self, shape, dtype=None):
        return K.constant(self.b ,shape = shape, dtype=dtype)
    
    def fit(self, X, y, epochs=100, learning_rate=0.1, alternate_num = 5): #Xは4階テンソル（データの個数）＊（企業データ）＊（イベント）＊（センチメント）
        
        if len(y.shape) == 1:
            self.y = y.reshape(-1, 1)
        else:
            self.y = y
        
        #交互最適を止めるための検証用データ作成
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=1)
        self.val_score = -float('inf')
        
        #企業データの特徴量の次元数
        feature_len_train = self.X_train.shape[0]
        feature_len_com = X.shape[1] 
        feature_len_eve = X.shape[2]
        feature_len_sen = X.shape[3]
        
        # それぞれのデータの係数
        self.W_com = np.random.randn(feature_len_com, 1)
        self.W_eve = np.random.randn(feature_len_eve, 1)
        self.W_sen = np.random.randn(feature_len_sen, 1)
        self.b = np.random.randn(1, 1)
        
#         self.W_com = np.array([[1], [2], [-1]])
#         self.W_eve = np.array([[-1], [1], [1]])
#         self.W_sen = np.array([[3], [1], [3]])
#         self.b = np.array([[2]])
        
#         self.W_com = np.array([[1.01], [3.], [-1.01]])
#         self.W_eve = np.array([[-2.01], [1.01], [1.01]])
#         self.W_sen = np.array([[3.01], [1.01], [6.01]])
#         self.b = np.array([[2.01]])
        
        
        # 後のfor文で数字とそれぞれの変数を認識するための辞書
        dic_len = {1:feature_len_com, 2:feature_len_eve, 3:feature_len_sen}
        dic_W = {1:self.W_com, 2:self.W_eve, 3:self.W_sen} 
        dic_Kernel = {1:self.mk_kernel_com, 2:self.mk_kernel_eve, 3:self.mk_kernel_sen}
        list_W = ['com', 'eve', 'sen']
        
        for num in range(alternate_num):
            print(str(num+1)+"回目")
            for m in [1,2,3]: # １：企業、２：イベント、３：センチメント
                print()
                print('\n'+['企業', 'イベント', 'センチメント'][m-1])
                exec("self.beta = np.array(tf.math.reduce_sum(tf.math.square(self.W_{0})))*np.array(tf.math.reduce_sum(tf.math.square(self.W_{1})))".format(list_W[2-m//3], list_W[1-m//2]))
                exec("self.model = keras.Sequential([layers.Dense(1, input_shape=[dic_len[m]], kernel_regularizer=keras.regularizers.l2(0.25*self.beta), kernel_initializer = self.mk_kernel_{}, bias_initializer = self.mk_bias)])".format(list_W[m-1]))

                self.model.compile(loss=CustomLoss(C = self.C, epsilon = self.epsilon), 
                                   optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False))
#                 self.model.compile(loss=CustomLoss(C = self.C, epsilon = self.epsilon), 
#                                     optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06))
                
    
                early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
                
                # n_mode_product展開でxを取得
                #self.X = n_mode_product(n_mode_product(self.X_train, dic_W[3-m//3].T, 3-m//3), dic_W[2-m//2].T, 2-m//2)
                exec("self.X = n_mode_product(n_mode_product(self.X_train, self.W_{0}.T, 3-m//3), self.W_{1}.T, 2-m//2)".format(list_W[2-m//3], list_W[1-m//2]))
                #exec("self.X = n_mode_product(n_mode_product(self.X_train, self.W_{0}, 3-m//3), self.W_{1}, 2-m//2)".format(list_W[2-m//3], list_W[1-m//2]))
                self.X = tf.reshape(self.X, [feature_len_train, dic_len[m]])
                
                self.model.fit(
                    self.X, self.y_train, validation_split = 0.2,
                    epochs=epochs, verbose=0, callbacks=[early_stop, PrintDot()])
                
                #dic_W[m] = self.model.layers[0].get_weights()[0]
                exec("self.W_{} = self.model.layers[0].get_weights()[0]".format(list_W[m-1]))
                self.b = self.model.layers[0].get_weights()[1]
                print('\n企業 : {}'.format(self.W_com))
                print('イベント : {}'.format(self.W_eve))
                print('センチメント : {}'.format(self.W_sen))
                print('intercept : {}'.format(self.b))
                print('='*50)
            #交互最適化を止めるか判断
            if self.score(self.X_test, self.y_test) -self.val_score<0:
                self.num = num
                return self
            self.val_score = self.score(self.X_test, self.y_test)
        self.num = num
        return self
    
    def predict(self, X, y=None):
        # tf.transpose()で置換されている
        y_pred = n_mode_product(n_mode_product(n_mode_product(X, tf.transpose(self.W_com), 1), tf.transpose(self.W_eve), 2), tf.transpose(self.W_sen), 3) + self.b
        y_pred = tf.reshape(y_pred,[len(X), 1]).numpy()
        
        return y_pred
    
    def coef_(self):
        result_W_com = self.W_com
        result_W_eve = self.W_eve
        result_W_sen = self.W_sen
        
        return result_W_com, result_W_eve, result_W_sen
    
    def intercept_(self):
        result_b = self.b
        
        return result_b
    
    def score(self, X, y):
        
        
        if len(y.shape) == 1:
            print('変換')
            y = y.reshape(-1, 1)
        # tf.transpose()で置換されている
        y_pred = n_mode_product(n_mode_product(n_mode_product(X, tf.transpose(self.W_com), 1), tf.transpose(self.W_eve), 2), tf.transpose(self.W_sen), 3) + self.b
        y_pred = tf.reshape(y_pred,[len(X), 1]).numpy()
        u = ((y_pred - y)**2).sum()
        v = ((y - y.mean())**2).sum()
        R2 = 1 - u/v
        
        return R2


### wから作る

f(x) = X *1 W_com *2 W_eve *3 W_sen + b

In [290]:
W_com = np.array([[1], [2], [-1]])
W_eve = np.array([[-1], [1], [1]])
W_sen = np.array([[3], [1], [3]])
b = np.array([2])

In [291]:
b.shape

(1,)

In [292]:
X = np.random.rand(10000,3, 3, 3)
X_test = np.random.rand(1000,3, 3, 3)
X

array([[[[0.46727418, 0.82308725, 0.76695345],
         [0.59041035, 0.62583251, 0.11700939],
         [0.51670005, 0.06368433, 0.83171686]],

        [[0.38436136, 0.67732527, 0.5632078 ],
         [0.52140161, 0.71171206, 0.11911258],
         [0.55760951, 0.12346225, 0.69031042]],

        [[0.10026121, 0.60995987, 0.33167396],
         [0.38679694, 0.2707384 , 0.78244132],
         [0.70302583, 0.29367038, 0.89497096]]],


       [[[0.81260606, 0.42329464, 0.63445989],
         [0.61195595, 0.41260803, 0.86610663],
         [0.9271983 , 0.68466512, 0.71832645]],

        [[0.91434831, 0.60667804, 0.05218643],
         [0.5066414 , 0.95323605, 0.85679522],
         [0.44987098, 0.25113616, 0.11328111]],

        [[0.16530077, 0.65275635, 0.83670162],
         [0.00902884, 0.40150153, 0.36680766],
         [0.88867936, 0.16628017, 0.342364  ]]],


       [[[0.43645178, 0.62672615, 0.3524108 ],
         [0.41655778, 0.03446641, 0.93300786],
         [0.38443829, 0.0444342 , 0.66205932

In [293]:
X = np.random.randn(10000,3, 3, 3)
X_test = np.random.randn(1000,3, 3, 3)
X

array([[[[-1.77013536,  0.93944555, -1.36001516],
         [ 1.04487607, -0.58617662,  1.49078181],
         [-0.30739842,  1.22528415,  2.53485131]],

        [[ 1.16650427,  0.41106613, -0.77305222],
         [ 0.26718732,  1.79922436, -0.37205206],
         [-1.31386718, -0.26460111,  1.08276346]],

        [[-0.34780875, -1.59095638,  0.06563218],
         [ 0.50504876, -1.97658164, -0.54191218],
         [ 0.06784045,  1.1033713 , -0.43117381]]],


       [[[-0.55373473,  1.09084656,  0.05952784],
         [-0.31612623, -0.97154056,  0.21965438],
         [ 0.31431571,  0.29792044, -1.50298402]],

        [[-1.22344677,  0.65466158, -1.03101151],
         [ 0.08889436, -1.57135595,  0.62026339],
         [-0.04381645, -0.26564528,  0.84989437]],

        [[-0.22690733,  0.88049278,  1.16258175],
         [ 1.19075458, -0.19628118, -0.33583397],
         [-0.72509362, -1.28045936, -0.58375214]]],


       [[[-0.88882127,  1.3238208 ,  0.7027708 ],
         [-1.09261492,  0.54450034

In [294]:
y = (n_mode_product(n_mode_product(n_mode_product(X, W_com.T, 1), W_eve.T, 2), W_sen.T, 3) + b).numpy().reshape(len(X), 1)
y_test = (n_mode_product(n_mode_product(n_mode_product(X_test, W_com.T, 1), W_eve.T, 2), W_sen.T, 3) + b).numpy().reshape(len(X_test), 1)
y

array([[ 22.88635167],
       [ 22.02360358],
       [-36.58233333],
       ...,
       [-15.15474354],
       [ 30.39386726],
       [-18.36089633]])

In [322]:
model = TeSIA(epsilon=0.2, C=10)

In [323]:
model.fit(X, y, epochs = 100, alternate_num = 1)

1回目


企業

..........
企業 : [[ 0.97125655]
 [ 1.9412566 ]
 [-0.97051656]]
イベント : [[-1.01]
 [ 1.01]
 [ 1.01]]
センチメント : [[3.01]
 [1.01]
 [3.01]]
intercept : [1.9888719]


イベント

......
企業 : [[ 0.97125655]
 [ 1.9412566 ]
 [-0.97051656]]
イベント : [[-1.0126275]
 [ 1.009614 ]
 [ 1.011185 ]]
センチメント : [[3.01]
 [1.01]
 [3.01]]
intercept : [1.9915166]


センチメント

.......
企業 : [[ 0.97125655]
 [ 1.9412566 ]
 [-0.97051656]]
イベント : [[-1.0126275]
 [ 1.009614 ]
 [ 1.011185 ]]
センチメント : [[3.0094352]
 [1.001969 ]
 [3.0095346]]
intercept : [1.9906368]


In [324]:
# テストデータの精度を計算
print("テストデータにフィット")
print("テストデータの精度 =", model.score(X_test, y_test))
print()
print("※参考")
print("訓練データの精度 =", model.score(X, y))

テストデータにフィット
テストデータの精度 = 0.9997616373687123

※参考
訓練データの精度 = 0.999760987094357


In [368]:
result = pd.DataFrame(index = ['W_com_1', 'W_com_2', 'W_com_3', 'W_eve_1', 'W_eve_2', 'W_eve_3', 'W_sen_1', 'W_sen_2', 'W_sen_3', 'int', 'train_score', 'test_score', 'alternate_num'])
for c in [100000000, 10000000, 1000000, 100000, 10000, 1000, 100, 10, 1]:
    model = TeSIA(epsilon=0.2, C=c)
    model.fit(X, y, epochs = 100, alternate_num = 100)
    result[str(c)] = model.W_com.reshape(3).tolist() + model.W_eve.reshape(3).tolist() + model.W_sen.reshape(3).tolist() + model.b.tolist() + [model.score(X, y)] + [model.score(X_test, y_test)] + [model.num]

1回目


企業

..............
企業 : [[ 0.7014854 ]
 [ 1.8396854 ]
 [-0.83854353]]
イベント : [[ 0.19395332]
 [ 1.0630446 ]
 [-0.28716936]]
センチメント : [[ 0.45530264]
 [-0.67583769]
 [ 0.18331909]]
intercept : [1.0314165]


イベント

............
企業 : [[ 0.7014854 ]
 [ 1.8396854 ]
 [-0.83854353]]
イベント : [[-2.2674327]
 [ 2.4565353]
 [ 2.0741858]]
センチメント : [[ 0.45530264]
 [-0.67583769]
 [ 0.18331909]]
intercept : [1.6322637]


センチメント

..........
企業 : [[ 0.7014854 ]
 [ 1.8396854 ]
 [-0.83854353]]
イベント : [[-2.2674327]
 [ 2.4565353]
 [ 2.0741858]]
センチメント : [[1.5371447]
 [0.5196919]
 [1.4684252]]
intercept : [1.9828243]
2回目


企業

........
企業 : [[ 0.87223774]
 [ 1.6822882 ]
 [-0.82485795]]
イベント : [[-2.2674327]
 [ 2.4565353]
 [ 2.0741858]]
センチメント : [[1.5371447]
 [0.5196919]
 [1.4684252]]
intercept : [1.9450735]


イベント

......
企業 : [[ 0.87223774]
 [ 1.6822882 ]
 [-0.82485795]]
イベント : [[-2.3954585]
 [ 2.38059  ]
 [ 2.3309727]]
センチメント : [[1.5371447]
 [0.5196919]
 [1.4684252]]
intercept : [1.967094]


センチメント

.....


..........
企業 : [[ 0.16866407]
 [ 0.00810028]
 [-0.37692997]]
イベント : [[-6.1470847]
 [ 5.6427593]
 [ 6.008075 ]]
センチメント : [[ 0.03838932]
 [-0.58780641]
 [ 0.8266106 ]]
intercept : [1.7587819]


センチメント

.............
企業 : [[ 0.16866407]
 [ 0.00810028]
 [-0.37692997]]
イベント : [[-6.1470847]
 [ 5.6427593]
 [ 6.008075 ]]
センチメント : [[1.4613658 ]
 [0.84305334]
 [1.8015023 ]]
intercept : [1.2547443]
2回目


企業

......
企業 : [[ 0.26042643]
 [ 0.5818339 ]
 [-0.29759163]]
イベント : [[-6.1470847]
 [ 5.6427593]
 [ 6.008075 ]]
センチメント : [[1.4613658 ]
 [0.84305334]
 [1.8015023 ]]
intercept : [1.9652941]


イベント

..............
企業 : [[ 0.26042643]
 [ 0.5818339 ]
 [-0.29759163]]
イベント : [[-5.8376174]
 [ 6.2451353]
 [ 5.9707723]]
センチメント : [[1.4613658 ]
 [0.84305334]
 [1.8015023 ]]
intercept : [1.9903934]


センチメント

.......
企業 : [[ 0.26042643]
 [ 0.5818339 ]
 [-0.29759163]]
イベント : [[-5.8376174]
 [ 6.2451353]
 [ 5.9707723]]
センチメント : [[1.7437466]
 [0.5999513]
 [1.7559855]]
intercept : [2.1210177]
3回目


企業

...........


......................
企業 : [[ 0.4033717 ]
 [ 0.80647296]
 [-0.40405285]]
イベント : [[ 2.8281338]
 [-2.8366084]
 [-2.8351145]]
センチメント : [[-2.626897  ]
 [-0.88144404]
 [-2.619391  ]]
intercept : [2.0134575]


センチメント

......
企業 : [[ 0.4033717 ]
 [ 0.80647296]
 [-0.40405285]]
イベント : [[ 2.8281338]
 [-2.8366084]
 [-2.8351145]]
センチメント : [[-2.6206062]
 [-0.8759569]
 [-2.6183383]]
intercept : [2.007878]
8回目


企業

.......
企業 : [[ 0.40396392]
 [ 0.80902636]
 [-0.40589043]]
イベント : [[ 2.8281338]
 [-2.8366084]
 [-2.8351145]]
センチメント : [[-2.6206062]
 [-0.8759569]
 [-2.6183383]]
intercept : [1.9760429]


イベント

........
企業 : [[ 0.40396392]
 [ 0.80902636]
 [-0.40589043]]
イベント : [[ 2.8218327]
 [-2.8262246]
 [-2.8308659]]
センチメント : [[-2.6206062]
 [-0.8759569]
 [-2.6183383]]
intercept : [1.9620018]


センチメント

......
企業 : [[ 0.40396392]
 [ 0.80902636]
 [-0.40589043]]
イベント : [[ 2.8218327]
 [-2.8262246]
 [-2.8308659]]
センチメント : [[-2.6224868 ]
 [-0.87794083]
 [-2.626993  ]]
intercept : [1.9861286]
9回目


企業

.......


.........
企業 : [[ 1.5212424]
 [ 2.91066  ]
 [-1.4612436]]
イベント : [[-0.45364696]
 [ 0.45442706]
 [ 0.43857014]]
センチメント : [[4.5212183]
 [1.6290412]
 [4.4959674]]
intercept : [1.8516189]


センチメント

.........
企業 : [[ 1.5212424]
 [ 2.91066  ]
 [-1.4612436]]
イベント : [[-0.45364696]
 [ 0.45442706]
 [ 0.43857014]]
センチメント : [[4.5514445]
 [1.4802564]
 [4.5875506]]
intercept : [1.9656304]
3回目


企業

........
企業 : [[ 1.4640754]
 [ 2.9468641]
 [-1.461105 ]]
イベント : [[-0.45364696]
 [ 0.45442706]
 [ 0.43857014]]
センチメント : [[4.5514445]
 [1.4802564]
 [4.5875506]]
intercept : [2.0044615]


イベント

......
企業 : [[ 1.4640754]
 [ 2.9468641]
 [-1.461105 ]]
イベント : [[-0.44775563]
 [ 0.44766438]
 [ 0.44890863]]
センチメント : [[4.5514445]
 [1.4802564]
 [4.5875506]]
intercept : [1.9991045]


センチメント

......
企業 : [[ 1.4640754]
 [ 2.9468641]
 [-1.461105 ]]
イベント : [[-0.44775563]
 [ 0.44766438]
 [ 0.44890863]]
センチメント : [[4.5520606]
 [1.5214597]
 [4.549511 ]]
intercept : [2.0013342]
4回目


企業

.............
企業 : [[ 1.4683286]
 [ 2.


..........
企業 : [[-0.38563624]
 [-0.69168156]
 [ 0.36576003]]
イベント : [[-0.95678526]
 [ 0.9819029 ]
 [ 0.94583493]]
センチメント : [[-8.691667 ]
 [-2.9841485]
 [-8.635721 ]]
intercept : [1.9739809]
1回目


企業

......
企業 : [[ 1.2635005]
 [ 2.718243 ]
 [-1.2084125]]
イベント : [[-0.49087972]
 [ 0.54544272]
 [ 0.43940888]]
センチメント : [[ 3.16075907]
 [-0.46433469]
 [ 0.38885491]]
intercept : [1.8208764]


イベント

.......
企業 : [[ 1.2635005]
 [ 2.718243 ]
 [-1.2084125]]
イベント : [[-0.45216227]
 [ 0.6214314 ]
 [ 0.5449106 ]]
センチメント : [[ 3.16075907]
 [-0.46433469]
 [ 0.38885491]]
intercept : [0.93401325]


センチメント

......
企業 : [[ 1.2635005]
 [ 2.718243 ]
 [-1.2084125]]
イベント : [[-0.45216227]
 [ 0.6214314 ]
 [ 0.5449106 ]]
センチメント : [[2.962081 ]
 [1.1427326]
 [2.7064981]]
intercept : [1.9829712]
2回目


企業

................
企業 : [[ 1.3721727]
 [ 2.5767522]
 [-1.2150346]]
イベント : [[-0.45216227]
 [ 0.6214314 ]
 [ 0.5449106 ]]
センチメント : [[2.962081 ]
 [1.1427326]
 [2.7064981]]
intercept : [2.31445]


イベント

........
企業 : [[

In [370]:
result#.to_csv('TeSIA_人工データ&初期点ランダム(標準正規分布).csv')

,100000000,10000000,1000000,100000,10000,1000,100,10,1
W_com_1,0.836645,-1.785803,0.278168,0.405679,4.231602,1.463247,3.326592,-0.385636,1.147810
W_com_2,1.674911,-3.581582,0.558411,0.807709,8.491730,2.931994,6.738735,-0.691682,2.502959
W_com_3,-0.835262,1.786066,-0.278481,-0.402792,-4.227047,-1.466685,-3.343646,0.365760,-1.351930
W_eve_1,-2.349527,1.691113,-6.123572,2.827592,0.598034,-0.449177,-1.956284,-0.956785,-0.690513
W_eve_2,2.352010,-1.687040,6.116592,-2.823149,-0.597605,0.448650,1.995610,0.981903,0.664655
W_eve_3,2.347491,-1.685128,6.132586,-2.826355,-0.597631,0.449752,1.965383,0.945835,0.507967
W_sen_1,1.522448,0.993626,1.759713,-2.621075,-1.180937,4.544112,0.503196,-8.691667,2.825396
W_sen_2,0.512171,0.333384,0.581246,-0.880109,-0.396624,1.513451,0.141147,-2.984149,0.931317
W_sen_3,1.527174,0.994650,1.755181,-2.630610,-1.181037,4.552215,0.437491,-8.635721,2.688351
int,2.039891,2.031024,1.993808,1.994961,2.001221,1.998050,1.883127,1.973981,1.968194


In [304]:
y_test

array([[  7.4658402 ],
       [ -7.81948455],
       [ 11.77095558],
       [ 24.47756536],
       [  8.87719306],
       [ -7.64193419],
       [ 11.68228344],
       [  4.83248935],
       [  5.85938452],
       [-16.0425182 ],
       [-33.28036698],
       [-28.31036569],
       [ -6.96106956],
       [ 19.6184926 ],
       [-27.69584538],
       [-39.59309686],
       [ 10.21540914],
       [ 20.29985222],
       [ 17.27093151],
       [  9.48212233],
       [ 26.27074655],
       [ 26.75438155],
       [ 36.76582143],
       [  0.91254477],
       [ 19.22641199],
       [-35.25130615],
       [ 26.00597916],
       [  1.1190643 ],
       [ 18.73510139],
       [  6.5138716 ],
       [-34.0434476 ],
       [ -6.57066128],
       [-25.51800829],
       [ 10.82190431],
       [ -2.32569453],
       [  1.09866557],
       [ 25.45665907],
       [-12.79522624],
       [ -2.40026841],
       [  1.12959882],
       [ -3.98315308],
       [-20.28935939],
       [-31.63104436],
       [-19

In [305]:
model.predict(X_test)

array([[1.54487  ],
       [1.5422409],
       [1.5431342],
       [1.5449175],
       [1.5401745],
       [1.5405548],
       [1.5422207],
       [1.5466899],
       [1.5413256],
       [1.5388805],
       [1.5364765],
       [1.5378152],
       [1.5400531],
       [1.5458524],
       [1.5382189],
       [1.5356128],
       [1.5414977],
       [1.5434991],
       [1.5446502],
       [1.5391104],
       [1.5468459],
       [1.5465721],
       [1.5476816],
       [1.5432938],
       [1.5435171],
       [1.5346235],
       [1.544179 ],
       [1.5371022],
       [1.5451908],
       [1.5408077],
       [1.5341532],
       [1.5404403],
       [1.5400732],
       [1.5409669],
       [1.5403361],
       [1.5387255],
       [1.5475336],
       [1.5378693],
       [1.5428805],
       [1.53984  ],
       [1.5372814],
       [1.5333318],
       [1.5394208],
       [1.540015 ],
       [1.5409969],
       [1.5432415],
       [1.5445472],
       [1.5447557],
       [1.5415195],
       [1.5442308],


In [306]:
y_test - model.predict(X_test)

array([[ 5.92097018e+00],
       [-9.36172541e+00],
       [ 1.02278214e+01],
       [ 2.29326479e+01],
       [ 7.33701858e+00],
       [-9.18248895e+00],
       [ 1.01400627e+01],
       [ 3.28579948e+00],
       [ 4.31805895e+00],
       [-1.75813987e+01],
       [-3.48168435e+01],
       [-2.98481809e+01],
       [-8.50112268e+00],
       [ 1.80726402e+01],
       [-2.92340642e+01],
       [-4.11287097e+01],
       [ 8.67391143e+00],
       [ 1.87563531e+01],
       [ 1.57262813e+01],
       [ 7.94301191e+00],
       [ 2.47239006e+01],
       [ 2.52078095e+01],
       [ 3.52181399e+01],
       [-6.30749064e-01],
       [ 1.76828949e+01],
       [-3.67859296e+01],
       [ 2.44618002e+01],
       [-4.18037922e-01],
       [ 1.71899106e+01],
       [ 4.97306388e+00],
       [-3.55776008e+01],
       [-8.11110160e+00],
       [-2.70580814e+01],
       [ 9.28093744e+00],
       [-3.86603066e+00],
       [-4.40059922e-01],
       [ 2.39091254e+01],
       [-1.43330956e+01],
       [-3.9

In [302]:
n_mode_product(n_mode_product(X, W_eve.T, 2), W_sen.T, 3)

<tf.Tensor: shape=(10000, 3, 1, 1), dtype=float64, numpy=
array([[[[ 23.37944586]],

        [[ -1.06470439]],

        [[  0.36368542]]],


       [[[ -4.13726654]],

        [[  8.81741905]],

        [[ -6.52603202]]],


       [[[ -4.22193338]],

        [[ -9.12898468]],

        [[ 16.10243059]]],


       ...,


       [[[-12.66831257]],

        [[  0.62323635]],

        [[  5.73290367]]],


       [[[  4.71768929]],

        [[  9.10368553]],

        [[ -5.46880692]]],


       [[[ -9.34778129]],

        [[ -1.34441532]],

        [[  8.3242844 ]]]])>

In [303]:
tf.reshape(n_mode_product(n_mode_product(X, W_eve.T, 2), W_sen.T, 3), [10000, 3])

<tf.Tensor: shape=(10000, 3), dtype=float64, numpy=
array([[ 23.37944586,  -1.06470439,   0.36368542],
       [ -4.13726654,   8.81741905,  -6.52603202],
       [ -4.22193338,  -9.12898468,  16.10243059],
       ...,
       [-12.66831257,   0.62323635,   5.73290367],
       [  4.71768929,   9.10368553,  -5.46880692],
       [ -9.34778129,  -1.34441532,   8.3242844 ]])>

In [ ]:
model.predict